In [186]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix,classification_report,balanced_accuracy_score

In [187]:
# Read the startup_data.csv file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path("./Resources/startup_data.csv")
)

# Review the DataFrame
df.head()

,Company_Name,Dependent-Company Status,year of founding,Age of company in years,Internet Activity Score,Short Description of company profile,Industry of company,Focus functions of company,Investors,Employee Count,...,Percent_skill_Data Science,Percent_skill_Business Strategy,Percent_skill_Product Management,Percent_skill_Sales,Percent_skill_Domain,Percent_skill_Law,Percent_skill_Consulting,Percent_skill_Finance,Percent_skill_Investment,Renown score
0,Company1,Success,No Info,No Info,-1.0,Video distribution,NaN,operation,KPCB Holdings|Draper Fisher Jurvetson (DFJ)|Kl...,3.0,...,0,0,0,0,0,0,0,0,0,0
1,Company2,Success,2011,3,125.0,NaN,Market Research|Marketing|Crowdfunding,"Marketing, sales",NaN,NaN,...,8.823529412,21.76470588,10.88235294,2.941176471,0,0,0,0,0,8
2,Company3,Success,2011,3,455.0,Event Data Analytics API,Analytics|Cloud Computing|Software Development,operations,TechStars|Streamlined Ventures|Amplify Partner...,14.0,...,3.846153846,17.09401709,9.401709402,0,2.777777778,0,0,0,0,9
3,Company4,Success,2009,5,-99.0,The most advanced analytics for mobile,Mobile|Analytics,Marketing & Sales,Michael Birch|Max Levchin|Sequoia Capital|Keit...,45.0,...,0,0,0,0,0,0,0,0,0,5
4,Company5,Success,2010,4,496.0,The Location-Based Marketing Platform,Analytics|Marketing|Enterprise Software,Marketing & Sales,DFJ Frontier|Draper Nexus Ventures|Gil Elbaz|A...,39.0,...,0,0,0,0,0,0,0,0,0,6


In [188]:
# Review the data types associated with the columns
df.dtypes

Company_Name                 object
Dependent-Company Status     object
year of founding             object
Age of company in years      object
Internet Activity Score     float64
                             ...   
Percent_skill_Law            object
Percent_skill_Consulting     object
Percent_skill_Finance        object
Percent_skill_Investment     object
Renown score                 object
Length: 116, dtype: object

In [189]:
# Keep relevant columns in the dataframe
startup_df = df[['Dependent-Company Status', # y target
                 'Age of company in years',
                 'Internet Activity Score', 
                 # 'Industry of company', 
                 'Focus functions of company', 
                 # 'Employee Count', 
                 # 'Has the team size grown', 
                 # 'Last Funding Date', 
                 'Last Funding Amount', 
                 'Country of company', 
                 # 'Continent of company', 
                 'Number of Investors in Seed', 
                 'Number of Investors in Angel and or VC', 
                 # 'Number of Co-founders', 
                 'Number of of advisors', 
                 'Team size Senior leadership', 
                 # 'Team size all employees', 
                 'Presence of a top angel or venture fund in previous round of investment', 
                 'Number of of repeat investors', 
                 # 'Number of  Sales Support material', 
                 # 'Worked in top companies', 
                 'Have been part of successful startups in the past?',
                 # 'Product or service company?', 
                 # 'Catering to product/service across verticals',
                 # 'Subscription based business', 
                 'Local or global player', 
                 'Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive',
                 # 'B2C or B2B venture?',
                 'Proprietary or patent position (competitive position)',
                 'Barriers of entry for the competitors',
                 # 'google page rank of company website',
                 'Disruptiveness of technology',
                 'Number of Direct competitors',
                 # 'Last round of funding received (in milionUSD)',
                 'Time to 1st investment (in months)', 
                 'Avg time to investment - average across all rounds, measured from previous investment']]
startup_df              

,Dependent-Company Status,Age of company in years,Internet Activity Score,Focus functions of company,Last Funding Amount,Country of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of of advisors,Team size Senior leadership,...,Number of of repeat investors,Have been part of successful startups in the past?,Local or global player,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Proprietary or patent position (competitive position),Barriers of entry for the competitors,Disruptiveness of technology,Number of Direct competitors,Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment"
0,Success,No Info,-1.0,operation,450000.0,United States,2,0,2,2,...,4,No,Global,Yes,No,Yes,Low,0,No Info,11.56
1,Success,3,125.0,"Marketing, sales",NaN,United States,5,0,0,4,...,0,Yes,Local,No,Yes,Yes,Medium,0,10,9
2,Success,3,455.0,operations,2350000.0,United States,15,0,0,7,...,0,No,Local,No,Yes,Yes,Medium,0,2,7.344444444
3,Success,5,-99.0,Marketing & Sales,10250000.0,United States,6,0,0,4,...,0,Yes,Local,No,Yes,Yes,Medium,2,1,8.7
4,Success,4,496.0,Marketing & Sales,5500000.0,United States,7,0,1,8,...,0,No,Local,Yes,Yes,Yes,Medium,0,13,9.822222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,Failed,3,-5.0,NaN,NaN,United States,0,0,0,1,...,0,No Info,local,No,No Info,Yes,No Info,No Info,No Info,No Info
468,Failed,NaN,NaN,consumer web,NaN,NaN,0,4,0,1,...,0,No Info,global,Yes,No Info,Yes,No Info,No Info,No Info,No Info
469,Failed,4,0.0,CAMPAIGN MANAGEMENT,NaN,United States,No Info,No Info,0,1,...,No Info,No Info,GLOBAL,No,No Info,Yes,No Info,No Info,No Info,No Info
470,Failed,5,0.0,SOCIALMEDIA CAMPAIGN,100000.0,United States,No Info,No Info,2,1,...,No Info,No Info,GLOBAL,No,No Info,Yes,No Info,No Info,No Info,No Info


### Rename Columns

In [190]:
startup_df.fillna(0)

,Dependent-Company Status,Age of company in years,Internet Activity Score,Focus functions of company,Last Funding Amount,Country of company,Number of Investors in Seed,Number of Investors in Angel and or VC,Number of of advisors,Team size Senior leadership,...,Number of of repeat investors,Have been part of successful startups in the past?,Local or global player,"Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive",Proprietary or patent position (competitive position),Barriers of entry for the competitors,Disruptiveness of technology,Number of Direct competitors,Time to 1st investment (in months),"Avg time to investment - average across all rounds, measured from previous investment"
0,Success,No Info,-1.0,operation,450000.0,United States,2,0,2,2,...,4,No,Global,Yes,No,Yes,Low,0,No Info,11.56
1,Success,3,125.0,"Marketing, sales",0.0,United States,5,0,0,4,...,0,Yes,Local,No,Yes,Yes,Medium,0,10,9
2,Success,3,455.0,operations,2350000.0,United States,15,0,0,7,...,0,No,Local,No,Yes,Yes,Medium,0,2,7.344444444
3,Success,5,-99.0,Marketing & Sales,10250000.0,United States,6,0,0,4,...,0,Yes,Local,No,Yes,Yes,Medium,2,1,8.7
4,Success,4,496.0,Marketing & Sales,5500000.0,United States,7,0,1,8,...,0,No,Local,Yes,Yes,Yes,Medium,0,13,9.822222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,Failed,3,-5.0,0,0.0,United States,0,0,0,1,...,0,No Info,local,No,No Info,Yes,No Info,No Info,No Info,No Info
468,Failed,0,0.0,consumer web,0.0,0,0,4,0,1,...,0,No Info,global,Yes,No Info,Yes,No Info,No Info,No Info,No Info
469,Failed,4,0.0,CAMPAIGN MANAGEMENT,0.0,United States,No Info,No Info,0,1,...,No Info,No Info,GLOBAL,No,No Info,Yes,No Info,No Info,No Info,No Info
470,Failed,5,0.0,SOCIALMEDIA CAMPAIGN,100000.0,United States,No Info,No Info,2,1,...,No Info,No Info,GLOBAL,No,No Info,Yes,No Info,No Info,No Info,No Info


In [191]:
# Create a list of categorical variables 
categorical_variables = list(startup_df.dtypes[startup_df.dtypes == "object"].index)

# Display the categorical variables list
categorical_variables

['Dependent-Company Status',
 'Age of company in years',
 'Focus functions of company',
 'Country of company',
 'Number of Investors in Seed',
 'Number of Investors in Angel and or VC',
 'Presence of a top angel or venture fund in previous round of investment',
 'Number of of repeat investors',
 'Have been part of successful startups in the past?',
 'Local or global player',
 'Capital intensive business e.g. e-commerce, Engineering products and operations can also cause a business to be capital intensive',
 'Proprietary or patent position (competitive position)',
 'Barriers of entry for the competitors',
 'Disruptiveness of technology',
 'Number of Direct competitors',
 'Time to 1st investment (in months)',
 'Avg time to investment - average across all rounds, measured from previous investment']

In [192]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [193]:
# Encode the categorical variables using OneHotEncoder
encoded_data = enc.fit_transform(startup_df[categorical_variables])
encoded_data

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [194]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

# Display the DataFrame
encoded_df.fillna(0)

,Dependent-Company Status_Failed,Dependent-Company Status_Success,Age of company in years_1,Age of company in years_10,Age of company in years_11,Age of company in years_12,Age of company in years_14,Age of company in years_15,Age of company in years_17,Age of company in years_2,...,"Avg time to investment - average across all rounds, measured from previous investment_9.077777778","Avg time to investment - average across all rounds, measured from previous investment_9.233333333","Avg time to investment - average across all rounds, measured from previous investment_9.322222222","Avg time to investment - average across all rounds, measured from previous investment_9.375","Avg time to investment - average across all rounds, measured from previous investment_9.45","Avg time to investment - average across all rounds, measured from previous investment_9.466666667","Avg time to investment - average across all rounds, measured from previous investment_9.688888889","Avg time to investment - average across all rounds, measured from previous investment_9.822222222","Avg time to investment - average across all rounds, measured from previous investment_96","Avg time to investment - average across all rounds, measured from previous investment_No Info"
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
468,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
470,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [195]:
# Create a DataFrame with the columnns containing numerical variables from the original dataset
numerical_variables_df = startup_df.drop(columns = categorical_variables)

# Review the DataFrame
numerical_variables_df.fillna(0)

,Internet Activity Score,Last Funding Amount,Number of of advisors,Team size Senior leadership
0,-1.0,450000.0,2,2
1,125.0,0.0,0,4
2,455.0,2350000.0,0,7
3,-99.0,10250000.0,0,4
4,496.0,5500000.0,1,8
...,...,...,...,...
467,-5.0,0.0,0,1
468,0.0,0.0,0,1
469,0.0,0.0,0,1
470,0.0,100000.0,2,1


In [196]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat(
    [
        numerical_variables_df,
        encoded_df
    ],
    axis=1
)

# Review the Dataframe
encoded_df.fillna(0)

,Internet Activity Score,Last Funding Amount,Number of of advisors,Team size Senior leadership,Dependent-Company Status_Failed,Dependent-Company Status_Success,Age of company in years_1,Age of company in years_10,Age of company in years_11,Age of company in years_12,...,"Avg time to investment - average across all rounds, measured from previous investment_9.077777778","Avg time to investment - average across all rounds, measured from previous investment_9.233333333","Avg time to investment - average across all rounds, measured from previous investment_9.322222222","Avg time to investment - average across all rounds, measured from previous investment_9.375","Avg time to investment - average across all rounds, measured from previous investment_9.45","Avg time to investment - average across all rounds, measured from previous investment_9.466666667","Avg time to investment - average across all rounds, measured from previous investment_9.688888889","Avg time to investment - average across all rounds, measured from previous investment_9.822222222","Avg time to investment - average across all rounds, measured from previous investment_96","Avg time to investment - average across all rounds, measured from previous investment_No Info"
0,-1.0,450000.0,2,2,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,125.0,0.0,0,4,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,455.0,2350000.0,0,7,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-99.0,10250000.0,0,4,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,496.0,5500000.0,1,8,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,-5.0,0.0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
468,0.0,0.0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,0.0,0.0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
470,0.0,100000.0,2,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [197]:
# Define the target set y using the Dependent-Company Status column
y = encoded_df["Dependent-Company Status_Success"]

# Display a sample of y
y

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
467    0.0
468    0.0
469    0.0
470    0.0
471    0.0
Name: Dependent-Company Status_Success, Length: 472, dtype: float64

In [198]:
# Define features set X by selecting all columns but Dependent-Company Status_Success
X = encoded_df.drop(columns=["Dependent-Company Status_Success"])

# Review the features DataFrame
X.fillna(0, inplace=True)
X

,Internet Activity Score,Last Funding Amount,Number of of advisors,Team size Senior leadership,Dependent-Company Status_Failed,Age of company in years_1,Age of company in years_10,Age of company in years_11,Age of company in years_12,Age of company in years_14,...,"Avg time to investment - average across all rounds, measured from previous investment_9.077777778","Avg time to investment - average across all rounds, measured from previous investment_9.233333333","Avg time to investment - average across all rounds, measured from previous investment_9.322222222","Avg time to investment - average across all rounds, measured from previous investment_9.375","Avg time to investment - average across all rounds, measured from previous investment_9.45","Avg time to investment - average across all rounds, measured from previous investment_9.466666667","Avg time to investment - average across all rounds, measured from previous investment_9.688888889","Avg time to investment - average across all rounds, measured from previous investment_9.822222222","Avg time to investment - average across all rounds, measured from previous investment_96","Avg time to investment - average across all rounds, measured from previous investment_No Info"
0,-1.0,450000.0,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,125.0,0.0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,455.0,2350000.0,0,7,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-99.0,10250000.0,0,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,496.0,5500000.0,1,8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,-5.0,0.0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
468,0.0,0.0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
469,0.0,0.0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
470,0.0,100000.0,2,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [199]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [200]:
# Review the distinct values from y
y_train.value_counts()

1.0    235
0.0    119
Name: Dependent-Company Status_Success, dtype: int64

In [201]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## after this section above (cleaning data) we input the information into the various different ML models below

# Neural Network Model (2 layers 1 output)

In [202]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

510

In [203]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [204]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2 

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

255

In [205]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2 

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

128

In [206]:
# Create the Sequential model instance
nn = Sequential()

In [207]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [208]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [209]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=1, activation="sigmoid"))

In [210]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 255)               130305    
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32768     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 163,202
Trainable params: 163,202
Non-trainable params: 0
_________________________________________________________________


In [211]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [212]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
12/12 [==============================] - 1s 2ms/step - loss: 0.5986 - accuracy: 0.6732
Epoch 2/50
12/12 [==============================] - 0s 3ms/step - loss: 0.1990 - accuracy: 0.9466
Epoch 3/50
12/12 [==============================] - 0s 2ms/step - loss: 0.0952 - accuracy: 0.9756
Epoch 4/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0458 - accuracy: 0.9959
Epoch 5/50
12/12 [==============================] - 0s 3ms/step - loss: 0.0197 - accuracy: 0.9982
Epoch 6/50
12/12 [==============================] - 0s 2ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 7/50
12/12 [==============================] - 0s 2ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 8/50
12/12 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 9/50
12/12 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 10/50
12/12 [==============================] - 0s 2ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 11/50
12/12 [

In [213]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

4/4 - 0s - loss: 0.1940 - accuracy: 0.9237
Loss: 0.19401174783706665, Accuracy: 0.9237288236618042


# RandomClassifier Model

In [214]:
# Import the RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Instantiate a RandomForestClassifier instance
model = RandomForestClassifier()

# Fit the traning data to the model
model.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [215]:
# Predict labels for original scaled testing features
y_pred = model.predict(X_test_scaled)

In [216]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, y_pred)

0.9895833333333333

In [217]:
# Print confusion matrix
confusion_matrix(y_test,y_pred)

array([[47,  1],
       [ 0, 70]])

In [218]:
# Print classification reports
print(f"Classifiction Report")
print(classification_report(y_pred, y_test))

Classifiction Report
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99        47
         1.0       1.00      0.99      0.99        71

    accuracy                           0.99       118
   macro avg       0.99      0.99      0.99       118
weighted avg       0.99      0.99      0.99       118



# Gaussian Process Classifier Model

In [219]:
from sklearn.gaussian_process import GaussianProcessClassifier

In [220]:
gaussian_model = GaussianProcessClassifier(max_iter_predict=1000,n_restarts_optimizer=3, random_state=1)

In [221]:
gaussian_model.fit(X_train_scaled, y_train)

GaussianProcessClassifier(max_iter_predict=1000, n_restarts_optimizer=3,
                          random_state=1)

In [222]:
y_pred = gaussian_model.predict(X_test_scaled)

In [223]:
balanced_accuracy_score(y_test, y_pred)

0.7550595238095239

In [224]:
confusion_matrix(y_test,y_pred)

array([[43,  5],
       [27, 43]])

In [225]:
print(f" Gaussian Classifiction Report")
print(classification_report(y_pred, y_test))

 Gaussian Classifiction Report
              precision    recall  f1-score   support

         0.0       0.90      0.61      0.73        70
         1.0       0.61      0.90      0.73        48

    accuracy                           0.73       118
   macro avg       0.76      0.76      0.73       118
weighted avg       0.78      0.73      0.73       118



# AdaBoost Model

In [226]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris

In [227]:
X, y = load_iris(return_X_y=True)

In [228]:
adaboost_model = AdaBoostClassifier(n_estimators=100)

In [229]:
scores = cross_val_score(adaboost_model, X, y, cv=5)

In [230]:
scores.mean()

0.9466666666666665

In [231]:
adaboost_model.fit(X_train_scaled, y_train)

AdaBoostClassifier(n_estimators=100)

In [232]:
y_pred = adaboost_model.predict(X_test_scaled)

In [233]:
balanced_accuracy_score(y_test, y_pred)

1.0

In [234]:
confusion_matrix(y_test,y_pred)

array([[48,  0],
       [ 0, 70]])

In [235]:
print(f" AdaBoost Classifiction Report")
print(classification_report(y_pred, y_test))

 AdaBoost Classifiction Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        48
         1.0       1.00      1.00      1.00        70

    accuracy                           1.00       118
   macro avg       1.00      1.00      1.00       118
weighted avg       1.00      1.00      1.00       118



## Classification Tree

In [236]:
# Import Modules
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image


In [237]:
# Split target column from dataset
y = df['Dependent-Company Status_Success']
X = df.drop(columns='Dependent-Company Status_Success')

# Set Index
X = X.set_index('id')

KeyError: 'Dependent-Company Status_Success'

In [ ]:
# Print first five entries for target
y[:5]

In [ ]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(X)

X.head()

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

In [ ]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [58]:
* Fitting the Decision Tree Model

NameError: name 'training_data' is not defined

In [59]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [60]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [224]:
* Making Predictions Using the Tree Model

NameError: name 'Question' is not defined

In [ ]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [222]:
# Displaying classification report
print(classification_report(y_test, predictions))

NameError: name 'Questions' is not defined

In [ ]:
 * Visualizing the Decision Tree

In [ ]:
# Create DOT data
dot_data = tree.export_graphviz(
    model, out_file=None, feature_names=X.columns, class_names=["0", "1"], filled=True
)

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())

In [ ]:
# Saving the tree as PDF
file_path = ("./Resources/startup_data.csv")
graph.write_pdf(file_path)

# Saving the tree as PNG
file_path = ("./Resources/startup_data.csv")
graph.write_png(file_path)